<p style="font-size:16px;">
    This is a testing notebook for trying the models before commiting to properly training them. Not only that, it allows for easier and more interactive visualizations once we have the trained weights.
</p>
<p style="font-size:16px;">
    It is divided into 3 main parts:
    <ul style="font-size:16px;">
        <li><b>Data Preparation:</b> Loading data, visualizing it to better understand the inputs and preparing it with the appropriate shape and data structure for the models</li>
        <li><b>Building Models:</b> Defining the models and testing small training instances to check they're working</li>
        <li><b>Visualizations:</b> Implementing various types of visualizations given the trained models to benefit from their insights</li>
    </ul>
</p>

# Data Preparation

<ul style="font-size:16px;">
    <li>Reading preprocessed data in an interleaved manner to extract data from both dancers separately</li>
    <li>Creating adjacencies by:
        <ol>
            <li>Initializing default skeleton for 29 joints</li>
            <li>Repeating the process for the second dancer</li>
            <li>Connecting every joint of a dancer to all joints on the other dancer (these are the ones we want to classify as existing or non-existing with the NRI model)</li>
        </ol>
    </li>
    <li>Visualizing both dancers with their default skeletons and also with the connecting edges</li>
    <li>Preparing batches with PyTorch tensors of shape (batches, length_of_sequences, number_of_joints_from_both_dancers, 3D)</li>
    <li>Creating training-validation split</li>
</ul>

In [ ]:
from IPython.core.debugger import set_trace
from glob import glob
import numpy as np

# Reading data coming from the pre-processing pipeline and creating both dancers
joint_poses = []
for file in glob('./data/*.npy'):
    interleaved_poses = np.load(file)
    poses_1 = interleaved_poses[0::2]
    poses_2 = interleaved_poses[1::2]
    
    joint_poses.append(np.concatenate((poses_1, poses_2), axis=1))
    print('Joint poses {} shape: {}\n'.format(file.split('/')[-1], joint_poses[-1].shape))

In [ ]:
# Building initial transposed edge index (adjacencies)
edge_index_t = [[0, 1], [0, 2], [0, 3], [1, 4], [2, 5], [3, 6], [4, 7], [5, 8], [6, 9], [7, 10], [7, 27], [8, 11],
                [8, 28], [9, 12], [9, 13], [9, 14], [12, 15], [13, 16], [14, 17], [15, 24], [16, 18], [17, 19],
                [18, 20], [19, 21], [20, 22], [21, 23], [22, 25], [23, 26]]

# Getting second person skeleton
n_joints = int(joint_poses[0].shape[1]/2)
init_skeleton_len = len(edge_index_t)
for edge_index in range(init_skeleton_len):
    edge_index_t.append([edge_index_t[edge_index][0]+n_joints, edge_index_t[edge_index][1]+n_joints])

# Saving skeletons for visualization
skeletons = edge_index_t.copy()

# Fully connecting the two people
for joint_1 in range(interleaved_poses.shape[1]):
    for joint_2 in range(interleaved_poses.shape[1]):
        edge_index_t.append([joint_1, joint_2+n_joints])

# Making graph undirected
full_skeleton_len = len(edge_index_t)
for edge_index in range(full_skeleton_len):
    edge_index_t.append([edge_index_t[edge_index][1], edge_index_t[edge_index][0]])

In [ ]:
# Visualizing joint skeletons for random frame
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

fig = plt.figure(figsize=(12, 4))

# Plotting dancers
ax1 = fig.add_subplot(121, projection="3d")
ax1.set_xlim([-1, 1])
ax1.set_ylim([-1, 1])
ax1.set_zlim([-1, 1])

rand_frame = 42
ax1.scatter(joint_poses[0][rand_frame, :n_joints, 2], joint_poses[0][rand_frame, :n_joints, 0], \
            -joint_poses[0][rand_frame, :n_joints, 1], color='red')
ax1.scatter(joint_poses[0][rand_frame, n_joints:, 2], joint_poses[0][rand_frame, n_joints:, 0], \
            -joint_poses[0][rand_frame, n_joints:, 1], color='blue')

for (start, end) in skeletons:
    xs = [joint_poses[0][rand_frame, start, 2], joint_poses[0][rand_frame, end, 2]]
    ys = [joint_poses[0][rand_frame, start, 0], joint_poses[0][rand_frame, end, 0]]
    zs = [-joint_poses[0][rand_frame, start, 1], -joint_poses[0][rand_frame, end, 1]]
    ax1.plot(xs, ys, zs, color='grey')

# Plotting dancers with fully connected joints
ax2 = fig.add_subplot(122, projection="3d")
ax2.set_xlim([-1, 1])
ax2.set_ylim([-1, 1])
ax2.set_zlim([-1, 1])

ax2.scatter(joint_poses[0][rand_frame, :n_joints, 2], joint_poses[0][rand_frame, :n_joints, 0], \
            -joint_poses[0][rand_frame, :n_joints, 1], color='red')
ax2.scatter(joint_poses[0][rand_frame, n_joints:, 2], joint_poses[0][rand_frame, n_joints:, 0], \
            -joint_poses[0][rand_frame, n_joints:, 1], color='blue')

for (start, end) in edge_index_t[:int(len(edge_index_t)/2)]:
    xs = [joint_poses[0][rand_frame, start, 2], joint_poses[0][rand_frame, end, 2]]
    ys = [joint_poses[0][rand_frame, start, 0], joint_poses[0][rand_frame, end, 0]]
    zs = [-joint_poses[0][rand_frame, start, 1], -joint_poses[0][rand_frame, end, 1]]
    ax2.plot(xs, ys, zs, color='grey')

plt.tight_layout()
plt.show()

In [ ]:
import torch

seq_len = 32
batches = []
choreo_lens = []

# # Building non-overlapping sequences
# for choreo in joint_poses:
#     choreo = torch.Tensor(choreo)

#     num_seqs = choreo.shape[0] // seq_len
#     batches.append(torch.stack([choreo[i*seq_len:(i+1)*seq_len] for i in range(num_seqs)]))
#     choreo_lens.append(batches[-1].size(0))

# Building overlapping sequences
for choreo in joint_poses:
    choreo = torch.Tensor(choreo)

    batches.append(torch.stack([choreo[i:i+seq_len] for i in range(len(choreo)-seq_len)]))
    choreo_lens.append(batches[-1].size(0))

batches = torch.cat(batches, dim=0)

# Balanced training-validation split
train_split = []
val_split = []
train_batches = []
val_batches = []
next_choreo = 0
for choreo_len in choreo_lens:
    
    train_split.append(int(0.85*choreo_len))
    val_split.append(choreo_len - train_split[-1])
    
    train_batches.append(batches[next_choreo : next_choreo + train_split[-1]])
    val_batches.append(batches[next_choreo + train_split[-1] : next_choreo + choreo_len])
    
    next_choreo += choreo_len

train_batches = torch.cat(train_batches, dim=0)
val_batches = torch.cat(val_batches, dim=0)

# Printing all the data structures created
print('Shape of tensor with all sequences: {}'.format(batches.shape))
print('Length of each choreography: {}\n'.format(choreo_lens))

print('Shape of training data with all sequences: {}'.format(train_batches.shape))
print('Length of each choreography in training dataset: {}\n'.format(train_split))

print('Shape of validation data with all sequences: {}'.format(val_batches.shape))
print('Length of each choreography in validation dataset: {}\n'.format(val_split))

# Bulding Models

<p style="font-size:16px;">The developed models were:</p>
<ul style="font-size:16px;">
    <li><b>NRI Variant:</b> Building a GCN variation of the Neural Relational Inference (NRI) model for the task of graph structure learning given sequences of movement (interactions of joints)</li>
    <li>Temporal model (?)</li>
</ul>

## NRI Variant

<p style="font-size:16px;">
    This model is a variant of the Neural Relational Inference (NRI) model, which itself is an extension of the traditional Variational Autoencoder (VAE). The primary objective of the original model is to study particles that move together in a system without prior knowledge of their underlying relationships. By analyzing their movements, the model aims to estimate a graph structure that connects these particles. In our context, the particles are represented by the joints of dancers. Although we know the physical connections between joints within a dancer's body, this information is insufficient to understand the artistic relationships between two dancers, such as how their joints move together or in opposition.
</p>

<p style="font-size:16px;">
Since we lack a target graph structure that correctly identifies which joints are virtually connected during a dance performance, and given that this graph can change over time within a performance (focusing on different body parts at different times), we employ self-supervising techniques.
</p>

<b style="font-size:18px;">Model Overview</b>

<p style="font-size:16px;">
The model consists of an encoder and a decoder, which play around with transforming node representations into edge representations and vice versa. This approach focuses on the dynamics of movements rather than fixed node embeddings. Since the encoder outputs edges (specifically, samples edges from the generated latent space), it is crucial to switch between these representations.
</p>

<p style="font-size:16px;">
Our implementation is similar to the NRI MLP-Encoder MLP-Decoder model, but with a couple of modifications:
</p>

<ul style="font-size:16px;">
    <li><b>Graph Convolutional Network (GCN):</b> We replaced some MLP layers with GCN layers to leverage the graph structure, improving the model's ability to capture relationships between joints. This change also helps us focus on a subset of edges that connect both dancers, rather than studying all particle relationships as in the original implementation. Additionally GCNs provide local feature aggregation and parameter sharing, important inductive biases for our context and resulting in enhanced generalization in a scenario with a "dynamic" (unknown) graph structure</li>
    <li><b>Predicting Sequences:</b> Since our data doesn't include velocity of the points, only their 3D position, the Markovian property doesn't hold. Therefore, to predict movement, we started reconstructing entire sequences</li>
    <li><b>Use of Modern Libraries:</b> We utilize PyTorch Geometric for its advanced features and ease of use</li>
</ul>

<p style="font-size:16px;">
By incorporating these modifications, our model maintains the core principles of the original NRI model while enhancing its ability to generalize and adapt to the dynamic nature of dance performances.
</p>

<b style="font-size:18px;">Final Architecture</b>

<ul style="font-size:16px;">
    <li>
        <b>Encoder</b>
        <ol style="font-size:16px;">
            <li>The encoder includes a GCN layer followed by a transformation of node representations into edge representations</li>
            <li>We then use an MLP layer, batch normalization and dropout</li>
            <li>After that, we convert edges back to nodes and apply another GCN layer</li>
            <li>Nodes are then transformed back into edges, followed by another MLP with a skip connection from the dropout layer</li>
            <li>And a final MLP layer outputs logits with two features representing the edge types (existing or non-existing)</li>
        </ol>
    </li>
    <li>
        <b>Decoder</b>
        <ol style="font-size:16px;">
            <li>Using the logits produced by the encoder, we hard sample a Gumbel-Softmax distribution. The idea is to approximate sampling in a continuous distribution and use Softmax to deal with the reparametrization trick, making the pipeline fully differentiable.</li>
            <li>With the newly sampled edge index in hand, the decoder starts by passing data into a GCN layer followed by a transformation of node representations into edge representations</li>
            <li>We then also use an MLP layer, batch normalization and dropout</li>
            <li>After that, we convert edges back to nodes and apply another GCN layer to get the reconstructed sequence</li>
        </ol>
    </li>
</ul>

</p>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Creating message passing matrices for receivers and senders - shape R^(E x N)
def message_passing_matrices(n_joints, edge_index):
    message_passing_in = torch.zeros((edge_index.size(1), n_joints))
    message_passing_out = torch.zeros((edge_index.size(1), n_joints))

    for j in range(edge_index.size(1)):
        message_passing_out[j, int(edge_index[0, j])] = 1.
        message_passing_in[j, int(edge_index[1, j])] = 1.

    return message_passing_in, message_passing_out

# NRI VAE auxiliar functions to change between nodes and edges
def node2edge(x, m_in, m_out):    
    receivers = torch.matmul(m_in, x)
    senders = torch.matmul(m_out, x)
    edges = torch.cat([senders, receivers], dim=1)
    
    return edges


def edge2node(x, m_in):
    incoming = torch.matmul(m_in.t(), x)
    
    return incoming / incoming.size(0)


# Gumbel-Softmax sampling function to allow for backpropagation with categorical distributions
def gumbel_softmax_sample(logits, temp, hard=False):
    y = F.gumbel_softmax(logits, tau=temp, hard=hard)
    
    return y


# Computing KL Divergence for categorical distribution
def gumbel_softmax_kl_divergence(logits, log_prior, batch_size):
    q_y = F.softmax(logits, dim=-1)
    kl_div = q_y * (F.log_softmax(logits, dim=-1) - log_prior)

    # Normalizing by the batch size and number of edges
    return kl_div.sum() / (batch_size * logits.size(0))


# Initializing reconstruction losses
nll_gaussian = nn.GaussianNLLLoss(reduction='sum') # Gaussian NLL
mse = nn.MSELoss(reduction='sum') # MSE

In [ ]:
import torch_geometric.nn as geo_nn
from torch_geometric.nn import GCNConv
from torch.nn import Linear, BatchNorm1d, Dropout

# Defining NRI encoder
class nri_encoder(nn.Module):
    def __init__(self, device, n_joints, edge_index_t, n_in, n_hid, n_out, do_prob=0.):
        super(nri_encoder, self).__init__()

        # Computing edge index given transposed edge index
        self.edge_index = torch.Tensor(edge_index_t).t().long().to(device)

        # Computing the message passing matrices
        self.m_in, self.m_out = message_passing_matrices(n_joints, self.edge_index)
        self.m_in = self.m_in.to(device)
        self.m_out = self.m_out.to(device)

        # Defining the network itself interleaving GCN and MLP layers
        self.conv1 = GCNConv(n_in, n_hid).to(device)
        
        self.mlp1 = Linear(n_hid*2, n_hid).to(device)
        self.bnorm1 = BatchNorm1d(n_hid).to(device)
        self.dropout1 = Dropout(do_prob).to(device)
        
        self.conv2 = GCNConv(n_hid, n_hid).to(device)
        
        self.mlp2 = Linear(n_hid*3, n_hid).to(device)
        self.bnorm2 = BatchNorm1d(n_hid).to(device)
        
        self.fc_out = Linear(n_hid, n_out).to(device)
        
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, Linear):
                nn.init.xavier_normal_(m.weight)
                m.bias.data.fill_(0.1)

    def forward(self, x):
        # Rearranging shapes: [num_seqs, num_timesteps, num_atoms, num_dims] -> [num_seqs, num_atoms, num_timesteps*num_dims]
        x = x.view(x.size(0), x.size(2), -1)

        # Forward pass interleaving GCN layers, operations to switch from nodes to edges or vice-versa, and MLP layers
        x = self.conv1(x, self.edge_index)
        x = F.relu(x)

        edge_x = [node2edge(x_samp, self.m_in, self.m_out) for x_samp in x]
        x = torch.stack(edge_x)
        
        x = self.mlp1(x)
        x = F.relu(x)

        x = x.permute(0, 2, 1)
        x = self.bnorm1(x)
        x = x.permute(0, 2, 1)
        
        x = self.dropout1(x)

        # Skip connection
        x_skip = x.clone()

        node_x = [edge2node(x_samp, self.m_in) for x_samp in x]
        x = torch.stack(node_x)
        
        x = self.conv2(x, self.edge_index)
        x = F.relu(x)
        
        edge_x = [node2edge(x_samp, self.m_in, self.m_out) for x_samp in x]
        x = torch.stack(edge_x)
        
        x = torch.cat((x, x_skip), dim=2)
        x = self.mlp2(x)
        x = F.relu(x)

        x = x.permute(0, 2, 1)
        x = self.bnorm2(x)
        x = x.permute(0, 2, 1)

        return self.fc_out(x)

In [ ]:
# Defining NRI decoder
class nri_decoder(nn.Module):
    def __init__(self, device, n_in, n_hid, n_out, do_prob=0.):
        super(nri_decoder, self).__init__()

        # Defining the network itself interleaving GCN and MLP layers
        self.conv1 = GCNConv(n_in, n_hid).to(device)
        
        self.mlp1 = Linear(n_hid*2, n_hid).to(device)
        self.bnorm1 = BatchNorm1d(n_hid).to(device)
        self.dropout1 = Dropout(do_prob).to(device)
        
        self.conv2 = GCNConv(n_hid, n_out).to(device)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, Linear):
                nn.init.xavier_normal_(m.weight)
                m.bias.data.fill_(0.1)

    def forward(self, x, edge_index, m_in, m_out):
        # Rearranging shapes: [num_seqs, num_timesteps, num_atoms, num_dims] -> [num_seqs, num_atoms, num_timesteps*num_dims]
        x = x.view(x.size(0), x.size(2), -1)

        # Forward pass interleaving GCN layers, operations to switch from nodes to edges or vice-versa, and MLP layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        edge_x = [node2edge(x_samp, m_in, m_out) for x_samp in x]
        x = torch.stack(edge_x)
        
        x = self.mlp1(x)
        x = F.relu(x)

        x = x.permute(0, 2, 1)
        x = self.bnorm1(x)
        x = x.permute(0, 2, 1)
        
        x = self.dropout1(x)

        node_x = [edge2node(x_samp, m_in) for x_samp in x]
        x = torch.stack(node_x)
        
        x = self.conv2(x, edge_index)

        return x

In [ ]:
# Defining NRI VAE
class nri_vae(nn.Module):
    def __init__(self, device, n_joints, edge_index_t, n_in, n_hid, edge_types, n_out, tau, hard, do_prob=0.):
        super(nri_vae, self).__init__()

        # Initializing encoder and decoder
        self.encoder = nri_encoder(device, n_joints, edge_index_t, n_in, n_hid, edge_types, do_prob)
        self.decoder = nri_decoder(device, n_in, n_hid, n_out, do_prob)

        # Saving variables that will be used by the forward pass
        self.device = device
        self.n_joints = n_joints
        
        self.tau = tau
        self.hard = hard
    
    def forward(self, x):
        # Computing logits for edges with encoder
        logits = self.encoder(x)

        # Sampling edge index classes using Gumbel-Softmax. Since we are using only two types of edges at the moment,
        # existent or non-existent, we create a newly sampled edge index for the decoder to use
        edge_index_classes = gumbel_softmax_sample(logits, tau, hard)
        edge_index_samp = torch.Tensor(edge_index_t).to(x.device)[torch.where(edge_index_classes[:, 1])[0]].t().long()

        # Creating message passing matrices for decoder newly sampled edge index
        decoder_m_in, decoder_m_out = message_passing_matrices(self.n_joints, edge_index_samp)
        decoder_m_in = decoder_m_in.to(self.device)
        decoder_m_out = decoder_m_out.to(self.device)

        # Reconstructing sequences using decoder
        recon_output = self.decoder(x, edge_index_samp, decoder_m_in, decoder_m_out)

        return logits, recon_output

In [ ]:
import time
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler

# Initializing all the hyperparameters and moving the required ones to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_batches = train_batches.to(device)
val_batches = val_batches.to(device)

batch_size = 8
train_batches_cumsum = np.cumsum(np.array(train_split) // batch_size)
val_batches_cumsum = np.cumsum(np.array(val_split) // batch_size)

seq_len_in = batches.size(1)
seq_len_out = 4
n_joints = batches.size(2)
dims = batches.size(3)

hidden_dims = 256
edge_types = 2

tau = 0.5
hard = True
dropout = 0.1
out_var = 5e-5

prior = [0.9, 0.1]
log_prior = torch.FloatTensor(np.log(prior)).unsqueeze(0).unsqueeze(0).to(device)

epochs = 500
lr = 5e-4
lr_decay = 50
gamma = 0.5

# Initializing model, optimizer and scheduler
model = nri_vae(device, n_joints, edge_index_t, seq_len_in*dims, hidden_dims, edge_types, seq_len_out*dims, tau, hard, dropout)

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=lr_decay, gamma=gamma)

# Initializing lists to save losses across iterations
kl_train = []
recon_train = []
loss_train = []
kl_val = []
recon_val = []
loss_val = []

# Initializing variables to save best model
best_val_loss = torch.inf
best_epoch = 0

# Model iteration function
def model_iteration(model, optimizer, scheduler, batches, batches_cumsum, beta, mode='train', recon_mode='nll'):
    t = time.time()
    
    kl_aux = []
    recon_aux = []
    loss_aux = []

    if mode == 'train':
        model.train()
    elif mode == 'val':
        model.eval()
    
    choreo_cumsum_idx = 0
    for idx in range(batches_cumsum[-1]):
        # Skipping last batch of a video, since the next batch belongs to the next video, not a delta_t of the movement
        if idx == batches_cumsum[choreo_cumsum_idx]:
            choreo_cumsum_idx += 1
            continue

        if mode == 'train':
            optimizer.zero_grad()
        
        batch = batches[idx*batch_size:(idx+1)*batch_size]
        # batch = batch.to(device)
        logits, recon_output = model(batch)

        kl_loss = gumbel_softmax_kl_divergence(logits, log_prior, batch_size)
        
        recon_output = recon_output.view(batch_size, seq_len_out, n_joints, dims)

        if recon_mode == 'nll':
            var_tensor = torch.full(recon_output.shape, out_var, device=device)
            recon_loss = nll_gaussian(recon_output, batches[(idx+1)*batch_size:(idx+2)*batch_size, :seq_len_out, :, :], var_tensor)    
        elif recon_mode == 'mse':
            recon_loss = mse(recon_output, batches[(idx+1)*batch_size:(idx+2)*batch_size, :seq_len_out, :, :])
            
        recon_loss = recon_loss / (recon_output.size(0) * recon_output.size(1) * recon_output.size(2))

        if recon_mode == 'nll':
            recon_coef = 0.0001
        elif recon_mode == 'mse':
            recon_coef = 1
            
        loss = 0.01*kl_loss + recon_coef*recon_loss
        # loss = recon_coef*recon_loss

        if mode == 'train':
            loss.backward()
            optimizer.step()

        kl_aux.append(0.01*kl_loss)
        recon_aux.append(recon_coef*recon_loss)    
        loss_aux.append(loss.data.item())

    kl_aux = torch.Tensor(kl_aux)
    recon_aux = torch.Tensor(recon_aux)
    loss_aux = torch.Tensor(loss_aux)
    tqdm.write(f'Epoch: {epoch + 1:04d}, '
               f'KL Loss ({mode}): {torch.mean(kl_aux):.4f}, '
               f'Reconstruction Loss ({mode}): {torch.mean(recon_aux):.4f}, '
               f'Combined Loss ({mode}): {torch.mean(loss_aux):.4f}, '
               f'time: {time.time() - t:.4f}s')

    if mode == 'train':
        scheduler.step()

    if mode == 'val':
        global best_val_loss
        global best_epoch
        
        if best_val_loss is torch.inf or torch.mean(loss_aux) < best_val_loss:    
            best_val_loss = torch.mean(loss_aux)
            best_epoch = epoch
            
            torch.save(model.state_dict(), 'best_weights/nri_parameters.pt')
            tqdm.write(f'Epoch: {epoch + 1:04d}, Saving best parameters!')

    del var_tensor
    torch.cuda.empty_cache()

    return kl_aux, recon_aux, loss_aux

# Training loop
# for epoch in tqdm(range(epochs), desc='Training Epochs'):
#     # Beta coefficient to handle KL-Divergence vanishing gradients and balance reconstruction loss
#     beta = epoch % int(epochs*0.1) / (epochs*0.1)

#     kl_aux, recon_aux, loss_aux = model_iteration(model, optimizer, scheduler, train_batches, train_batches_cumsum, beta, 'train', 'nll')
    
#     kl_train.append(torch.mean(kl_aux))
#     recon_train.append(torch.mean(recon_aux))
#     loss_train.append(torch.mean(loss_aux))

#     kl_aux, recon_aux, loss_aux = model_iteration(model, optimizer, scheduler, val_batches, val_batches_cumsum, beta, 'val', 'nll')
    
#     kl_val.append(torch.mean(kl_aux))
#     recon_val.append(torch.mean(recon_aux))
#     loss_val.append(torch.mean(loss_aux))

# print("Optimization Finished!")

In [ ]:
fig = plt.figure(figsize=(12, 6))

# Plotting training losses
ax1 = fig.add_subplot(231)
ax1.plot(kl_train, color='blue')

ax1 = fig.add_subplot(232)
ax1.plot(recon_train, color='blue')

ax1 = fig.add_subplot(233)
ax1.plot(loss_train, color='blue')

# Plotting validation losses
ax1 = fig.add_subplot(234)
ax1.plot(kl_val, color='orange')

ax1 = fig.add_subplot(235)
ax1.plot(recon_val, color='orange')

ax1 = fig.add_subplot(236)
ax1.plot(loss_val, color='orange')

# Adding column labels
fig.text(0.22, 0.96, 'KL Loss', ha='center', fontsize=14)
fig.text(0.53, 0.96, 'Reconstruction Loss', ha='center', fontsize=14)
fig.text(0.85, 0.96, 'Combined Loss', ha='center', fontsize=14)

# Adding row labels
fig.text(0.02, 0.73, 'Training', va='center', rotation='vertical', fontsize=14)
fig.text(0.02, 0.30, 'Validation', va='center', rotation='vertical', fontsize=14)

plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])
plt.show()

In [ ]:
# Counting number of trainable parameters to compare to the dataset size
print('Total number of trainable parameters: {}\n'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

for n, p in model.named_parameters():
    if p.requires_grad:
        print('Layer {} has {} trainbale parameters'.format(n, p.numel()))

# Visualizations

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Increasing animation memory limit
from matplotlib import rcParams
rcParams['animation.embed_limit'] = 100

# Animation function
def animation(sequence, skeleton=None, interval=100):
    fig = plt.figure(figsize=(16, 12))
    
    ax = fig.add_subplot(111, projection="3d")
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    ms = 70
    scatt1 = ax.scatter([], [], [], color='red', s=ms)
    scatt2 = ax.scatter([], [], [], color='blue', s=ms)

    if skeleton is not None:
        lw = 4
        lines = [ax.plot([], [], [], 'gray', linewidth=lw)[0] for _ in skeleton]

    sequence_x = sequence[:, :, 2]
    sequence_y = sequence[:, :, 0]
    sequence_z = -sequence[:, :, 1]
    
    def update(frame):
        
        scatt1._offsets3d = (sequence_x[frame, :n_joints], sequence_y[frame, :n_joints], sequence_z[frame, :n_joints])
        scatt2._offsets3d = (sequence_x[frame, n_joints:], sequence_y[frame, n_joints:], sequence_z[frame, n_joints:])
    
        if skeleton is not None:
            for line, (start, end) in zip(lines, skeleton):
                line.set_data([sequence_x[frame, start], sequence_x[frame, end]], [sequence_y[frame, start], sequence_y[frame, end]])
                line.set_3d_properties([sequence_z[frame, start], sequence_z[frame, end]])
            
            return scatt1, scatt2, *lines

        return scatt1, scatt2

    plt.close(fig)
    return FuncAnimation(fig, update, frames=range(len(sequence_x)), interval=interval, blit=False)

# ani = animation(train_batches[sequence], skeletons, interval=100)
# HTML(ani.to_jshtml())

## NRI Graph Structure Learning

In [ ]:
# Loading best model
model.load_state_dict(torch.load('best_weights/nri_parameters.pt'))

# Choosing sequence based on the selected video and frame, and preparing n_joints for plotting
choreo_idx = 0
offset = 0
if choreo_idx != 0:
    offset = np.cumsum(train_split)[choreo_idx-1]

frame = 0
sequence = frame // seq_len_in

n_joints = int(batches.size(2)/2)

# Getting predicted edges from a sequence
logits = model.encoder(train_batches[offset + sequence].unsqueeze(0)).squeeze(0)
edge_index_samp = np.array(edge_index_t)[torch.where(logits.argmax(dim=1))[0].cpu()]

# Getting reconstruction for next sequence
edge_index_samp_decoder = torch.Tensor(edge_index_t).to(device)[torch.where(logits.argmax(dim=1))[0]].t().long()
decoder_m_in, decoder_m_out = message_passing_matrices(n_joints*2, edge_index_samp_decoder)
decoder_m_in = decoder_m_in.to(device)
decoder_m_out = decoder_m_out.to(device)

recon_output = model.decoder(train_batches[offset + sequence].unsqueeze(0), \
                             edge_index_samp_decoder, decoder_m_in, decoder_m_out)
recon_output = recon_output.view(seq_len_out, n_joints*2, dims).cpu().detach().numpy()

In [ ]:
# Plotting dancers
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(121, projection="3d")
ax1.set_xlim([-1, 1])
ax1.set_ylim([-1, 1])
ax1.set_zlim([-1, 1])
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])

ax1.scatter(joint_poses[choreo_idx][frame, :n_joints, 2], joint_poses[choreo_idx][frame, :n_joints, 0], \
            -joint_poses[choreo_idx][frame, :n_joints, 1], color='red')
ax1.scatter(joint_poses[choreo_idx][frame, n_joints:, 2], joint_poses[choreo_idx][frame, n_joints:, 0], \
            -joint_poses[choreo_idx][frame, n_joints:, 1], color='blue')

for (start, end) in skeletons:
    xs = [joint_poses[choreo_idx][frame, start, 2], joint_poses[choreo_idx][frame, end, 2]]
    ys = [joint_poses[choreo_idx][frame, start, 0], joint_poses[choreo_idx][frame, end, 0]]
    zs = [-joint_poses[choreo_idx][frame, start, 1], -joint_poses[choreo_idx][frame, end, 1]]
    ax1.plot(xs, ys, zs, color='grey')

# Plotting dancers with sampled edges
ax2 = fig.add_subplot(122, projection="3d")
ax2.set_xlim([-1, 1])
ax2.set_ylim([-1, 1])
ax2.set_zlim([-1, 1])
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_zticks([])

ax2.scatter(joint_poses[choreo_idx][frame, :n_joints, 2], joint_poses[choreo_idx][frame, :n_joints, 0], \
            -joint_poses[choreo_idx][frame, :n_joints, 1], color='red')
ax2.scatter(joint_poses[choreo_idx][frame, n_joints:, 2], joint_poses[choreo_idx][frame, n_joints:, 0], \
            -joint_poses[choreo_idx][frame, n_joints:, 1], color='blue')

for (start, end) in edge_index_samp:
    xs = [joint_poses[choreo_idx][frame, start, 2], joint_poses[choreo_idx][frame, end, 2]]
    ys = [joint_poses[choreo_idx][frame, start, 0], joint_poses[choreo_idx][frame, end, 0]]
    zs = [-joint_poses[choreo_idx][frame, start, 1], -joint_poses[choreo_idx][frame, end, 1]]
    ax2.plot(xs, ys, zs, color='grey')

plt.tight_layout()
plt.show()

In [ ]:
from IPython.display import Video, display

# Getting dataset back to CPU
train_batches = train_batches.cpu()

# Evaluating the edge prediction by watching the dance sequence with and without the connections
ani = animation(train_batches[offset + sequence], skeletons, interval=100)
ani_html_no_edge_pred = ani.to_jshtml()

ani = animation(train_batches[offset + sequence], edge_index_samp, interval=100)
ani_html_edge_pred = ani.to_jshtml()

# Evaluating reconstruction by watching next dance sequence and the predicted one
ani = animation(train_batches[offset + sequence + 1], skeletons, interval=100)
ani_html_no_mov_pred = ani.to_jshtml()

ani = animation(recon_output, skeletons, interval=100)
ani_html_mov_pred = ani.to_jshtml()

display(HTML(f"""
<div style="display: flex; justify-content: space-around;">
    <div style="margin: 5px;">
        {ani_html_no_edge_pred}
    </div>
    <div style="margin: 5px;">
        {ani_html_edge_pred}
    </div>
</div>
<div style="display: flex; justify-content: space-around;">
    <div style="margin: 5px;">
        {ani_html_no_mov_pred}
    </div>
    <div style="margin: 5px;">
        {ani_html_mov_pred}
    </div>
</div>
"""))